In [1]:
from os.path import join
from os import walk
import pandas as pd
import numpy as np
import networkx as nx

ModuleNotFoundError: No module named 'networkx'

In [9]:
root_folder = r'.'

In [14]:
# final_folder = 'NoME&D'
# in_folder=join(root_folder,
#                final_folder,
#               )
in_folder = root_folder # join(root_folder, 'Data')
csv_files = [join(root, name)
             for root, dirs, files in walk(in_folder)
             for name in files
             if name.endswith('.csv')
             # if 'NP20250204' in name # This are the extended
             if 'mlvar_' in name # This are the limited
             # if 'luster' not in name
             # if 'participantsstatistics' not in name
            ]
# del(csv_files[-2])
len(csv_files), csv_files

mlvar_dict = {}
for csv_file in csv_files[:]:
    df = pd.read_csv(csv_file, 
                     # parse_dates=[2],
                     # sep=' ',
                     # skiprows=8,
                     # header=None,
                    )
    key = (csv_file[:-4]
           .split('\\')[-1]
           .split('_')[1]
          )
    # columns = [c
    #            for c in df.columns
    #            if not c.startswith('Unnamed:')
    #            # and not 'P' in c
    #           ][:]
    # df_mlvar = pd.DataFrame(columns=columns)
    # for i, row in df.iterrows():
    #     # print(key, row.dropna().values)
    #     # pd.DataFrame(row)#.dropna()
    #     # row.dropna()
    #     df_mlvar.loc[i, :] = row.dropna().values
    mlvar_dict[key] = df#_mlvar

dict_mlvar = {}
new_columns = ['n0',
               'n1',
               'similarity',
              ]
for key, df in mlvar_dict.items():
    columns = df.columns
    new_df = pd.DataFrame(columns=new_columns,
                         )
    if columns[0]=='v1':
        # df0 = df.iloc[:, :3]
        # df0.columns = new_columns
        # df1 = df.iloc[:, 3:].values
        # df1 = pd.DataFrame(columns=new_columns[-1:],
        #                    data=df1,
        #                   )
        # for column, index in zip(new_columns[:-1],
        #                          [1,0],
        #                         ):
        #     df1.insert(loc=0,
        #                column=column,
        #                value=(df0
        #                       .iloc[:,index]
        #                       .values
        #                      ),
        #               )
        # new_df = pd.concat([new_df,
        #                     df0,
        #                     df1,
        #                    ],
        #                    axis=0
        #                   )
        new_df = pd.DataFrame(columns=new_columns,
                              data=(df
                                    .iloc[:,[0,1,4]]
                                    .values
                                   ),
                             )
    else:
        # df.drop(df.columns[2],
        #         axis=1,
        #         inplace=True,
        #        )
        new_df = pd.DataFrame(columns=new_columns,
                              data=(df
                                    .iloc[:,[0,1,3]]
                                    .values
                                   ),
                             )
    dict_mlvar[key] = new_df

dict_mlvar

{'between':                    n0                 n1 similarity
 0          depression       hopelessness      0.425
 1              voices       hopelessness      0.014
 2              voices         depression     -0.027
 3             visions       hopelessness      0.015
 4             visions         depression      0.117
 5             visions             voices      0.421
 6             anxiety       hopelessness     -0.175
 7             anxiety         depression      0.741
 8             anxiety             voices      0.038
 9             anxiety            visions     -0.005
 10           paranoia       hopelessness     -0.255
 11           paranoia         depression      0.448
 12           paranoia             voices      0.201
 13           paranoia            visions     -0.247
 14           paranoia            anxiety      0.019
 15          delusions       hopelessness      0.118
 16          delusions         depression     -0.323
 17          delusions             

In [15]:
def GraphBuilding(nodes, 
                  edges,
                  directional=False,
                 ):
    if not directional:
        g = nx.Graph()
    else:
        g = nx.DiGraph()
    g.add_nodes_from(node for node in nodes.items())
    g.add_edges_from([(c0, 
                       c1,
                       # {'similarity': d['similarity'],
                       #  'distance': d['distance'],
                       # }
                       d,
                      )
                      for (c0, c1), d 
                      in edges.items()
                     ],
                    )
    return g

In [21]:
def StrengthCentrality(G, node, attribute):
    """
    Node strength is defined as the sum of the strengths (or weights) of all
    of the nodes edges.

    Parameters
    ----------
    G: graph
       A networkx graph
    node: object
       A node in the networkx graph
    attribute: object
       The edge attribute used to quantify node strength

    Returns
    -------
    node strength: (int, float)
    """
    output = 0.0
    for edge in G.edges(node):
        output += np.abs(G.get_edge_data(*edge)[attribute])
    return output

In [25]:
# direction = 'Symmetric'
seed=13
# (constructs_combinations,
# constructs) = ConstructCombinations(df_long)
# out_folder = r'.\NetworksPlots\mlVAR'
# data_folder= r'.\Results\mlVAR'
# plot_folder = 'Plots'
# data_folder = 'Data'
dict_graphs = {}
for key, df_mlvar in dict_mlvar.items():
    # constructs = list(df_mlvar['from'].value_counts().keys())
    constructs = sorted(df_mlvar
                  .iloc[:,:2]
                  .stack()
                  .value_counts()
                  .keys()
                 )
    # constructs
    constructs_combinations = df_mlvar.values
    # constructs_combinations

    edges = {(c0,c1):float(v) for c0,c1, v in constructs_combinations}

    nodes = {q:{#'color':spectrum(i/len(constructs)),
                      }
             for i, q 
             in enumerate(constructs)
            }

    display_plot = True
    # seed = 13
    fontsize_node = 9.5

    max_distance = max([d for d in edges.values()])

    edges = {k:{'similarity':v,
                # 'distance':1/v if,
                'absolute_similarity': abs(v),
               }
             for k,v
             in edges.items()
            }
    directional = True if key=='temporal' else False
    g = GraphBuilding(nodes,
                      edges,
                      directional=directional,
                     )

    betweenness = nx.betweenness_centrality(g,
                                            endpoints=True,
                                            weight='absolute_similarity',
                                           )
    # degree = nx.degree_centrality(g)
    closeness = nx.closeness_centrality(g,
                                        distance='absolute_similarity',
                                        wf_improved=True,
                                       )
    # eigenvector = nx.eigenvector_centrality(g,
    #                                         weight='distance',
    #                                         max_iter=1000,
    #                                        )

    strength = {node:StrengthCentrality(g,
                                        node,
                                        'absolute_similarity',
                                       )
                for node
                in g.nodes()
               }
    # print(nodes.keys())
    # fig, ax = GraphPlot(node_size_by=strength,
    #                     g=g,
    #                     plot_text=('Node size ~ Strength Centrality'
    #                                '\nEdges thickness ~ normalised fixed effects'
    #                                '\nEdge values = fixed effects'
    #                               ),
    #                     plot_title=('mlVAR Network\n'
    #                                 # 'mlVAR Network'
    #                                 # 'Weighted average from all participants '
    #                                 f'{key}'
    #                                ),
    #                     directional=directional,
    #                    )

    # color = dict(g.nodes.data('color'))
    nodes_centralities = {n:{name:c[n] 
                             for name, c
                             in [['betweenness', betweenness],
                                 ['strength', strength],
                                 # ['degree', degree],
                                 ['closeness', closeness],
                                 # ['color', color],
                                ]
                            } for n 
                          in nodes.keys()
                         }
    g = GraphBuilding(nodes_centralities,
                      edges,
                      directional=(True if key=='temporal' 
                                  else False),
                     )
    dict_graphs[key] = g
    # with open(join(root_folder,
    #                # data_folder,
    #                f'mlVAR_{key}_NetGraph.pkl',
    #               ),
    #           'wb',
    #          ) as f:
    #     joblib.dump(g, f)

    # for ext in ('png', 'pdf'): 
    #     out_path = join(root_folder,
    #                     plot_folder,
    #                     f'mlVAR_{key}_Py.{ext}',
    #                    )
    #     # out_path = join(out_folder, f'mlVAR_{key}_Py.{ext}')
    #     plt.savefig(out_path,
    #                 bbox_inches='tight',
    #                )
dict_graphs

{'between': <networkx.classes.graph.Graph at 0x28bd1bd9a30>,
 'contemporaneous': <networkx.classes.graph.Graph at 0x28bd1bef8a0>,
 'temporal': <networkx.classes.digraph.DiGraph at 0x28bd1b9ee90>}